In [29]:
#Distribution elaad
#inter arrival distributions and processing time

#old code # created elaad_env

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as stats
from scipy.stats import gamma
import numpy as np


In [30]:
file_path1 = r"/mnt/c/Users/s139501/CSV/open_transactions.csv"
file_path2 = r"/mnt/c/Users/s139501/CSV/open_metervalues.csv"

df1 = pd.read_csv(file_path1, delimiter=';', decimal=',')
df2 = pd.read_csv(file_path2, delimiter=';', decimal=',')

# Convert the columns to datetime
df1['UTCTransactionStart'] = pd.to_datetime(df1['UTCTransactionStart'], format='%d/%m/%Y %H:%M')
df1['UTCTransactionStop'] = pd.to_datetime(df1['UTCTransactionStop'], format='%d/%m/%Y %H:%M')

# Extract the hours and days
df1['ArrivalHour'] = df1['UTCTransactionStart'].dt.hour
df1['DepartureHour'] = df1['UTCTransactionStop'].dt.hour
df1['DayOfWeek'] = df1['UTCTransactionStart'].dt.dayofweek
df1['ArrivalMinute'] = df1['UTCTransactionStart'].dt.hour * 60 + df1['UTCTransactionStart'].dt.minute

# Segregate df1 into weekdays and weekends
df_weekdays = df1[df1['DayOfWeek'].between(0, 4)]
df_weekends = df1[df1['DayOfWeek'].between(5, 6)]

# Calculate the average number of arrivals per hour for weekdays
average_hourly_arrivals_weekdays = df_weekdays.groupby('ArrivalHour').size().reset_index(name='AverageArrivals')
average_hourly_arrivals_weekdays['AverageArrivals'] /= len(df_weekdays['DayOfWeek'].unique())

In [ ]:
# Plotting the average number of arrivals per hour for weekdays
plt.figure(figsize=(10, 6))
sns.barplot(data=average_hourly_arrivals_weekdays, x='ArrivalHour', y='AverageArrivals')
plt.title('Average Number of EV Arrivals per Hour on a Weekday')
plt.xlabel('Hour of the Day')
plt.ylabel('Average Number of Arrivals')
plt.xticks(range(0, 24))
plt.grid(axis='y', alpha=0.75)
plt.tight_layout()
plt.show()

In [ ]:
##PER SECOND##
# Sort data based on UTCTransactionStart
df1_sorted = df1.sort_values(by='UTCTransactionStart')

# Calculate time differences between consecutive transactions in seconds
df1_sorted['TimeDiff'] = df1_sorted['UTCTransactionStart'].diff().dt.total_seconds()

# Replace TimeDiff values of 0 or smaller with a small number (e.g., 0.001 seconds)
df1_sorted['TimeDiff'] = np.where(df1_sorted['TimeDiff'] <= 0, 0.001, df1_sorted['TimeDiff'])

# Drop the first row as it will have a NaN value for TimeDiff
df1_sorted = df1_sorted.dropna(subset=['TimeDiff'])

# Calculate the number of unique days in the dataset
num_days = df1_sorted['UTCTransactionStart'].dt.date.nunique()

# Calculate the frequency of transaction arrivals per day
time_diff_per_day = df1_sorted['TimeDiff'].value_counts().sort_index() / num_days

# Fit the gamma distribution to the 'TimeDiff' data, fixing the location at 0
params = stats.gamma.fit(df1_sorted['TimeDiff'], floc=0)

# Generate a range of values for which we will plot the gamma distribution
x = np.linspace(0, df1_sorted['TimeDiff'].quantile(0.95), 1000)

# Calculate the PDF of the fitted gamma distribution
pdf_fitted = stats.gamma.pdf(x, *params)

# Plot histogram and fitted distribution
plt.figure(figsize=(15, 6))
bin_size = 10
bins = int((df1_sorted['TimeDiff'].max() - df1_sorted['TimeDiff'].min()) / bin_size)
sns.histplot(df1_sorted['TimeDiff'], bins=bins, kde=False, stat='density')
plt.plot(x, pdf_fitted, label='Fitted Gamma Distribution')
plt.xlabel('Time Difference Between Transactions (Seconds)')
plt.ylabel('Normalized Frequency')
plt.title('Average Daily Frequency of Time Differences Between Transactions')
plt.xlim(0, df1_sorted['TimeDiff'].quantile(0.95))
plt.legend()
plt.show()



In [ ]:
# The emperical data distribution plot
plt.figure(figsize=(15, 6))
bin_size = 10  # Adjust the bin size for better granularity in seconds
bins = int((df1_sorted['TimeDiff'].max() - df1_sorted['TimeDiff'].min()) / bin_size)
sns.histplot(df1_sorted['TimeDiff'], bins=bins)
plt.xlabel('Time Difference Between Transactions (Seconds)')
plt.ylabel('Average Frequency per Day')
plt.title('Average Daily Frequency of Time Differences Between Transactions')
plt.xlim(0, df1_sorted['TimeDiff'].quantile(0.95))  # Limit x-axis to exclude rare extreme values if necessary
plt.show()

In [ ]:
##PER MINUTE A##
# Empirical distribution of time differences between transactions
# Sort data based on UTCTransactionStart
df1_sorted = df1.sort_values(by='UTCTransactionStart')

# Calculate time differences between consecutive transactions in minutes
df1_sorted['TimeDiff'] = df1_sorted['UTCTransactionStart'].diff().dt.total_seconds() / 60

# Sort and print the TimeDiff column in ascending order
print(df1_sorted['TimeDiff'].sort_values())

# Drop the first row as it will have a NaN value for TimeDiff
df1_sorted = df1_sorted.dropna(subset=['TimeDiff'])

# Calculate the number of unique days in the dataset
num_days = df1_sorted['UTCTransactionStart'].dt.date.nunique()

# Calculate the frequency of transaction arrivals per day
time_diff_per_day = df1_sorted['TimeDiff'].value_counts().sort_index() / num_days

#plot
plt.figure(figsize=(15, 6))
bin_size = 1  # The bin size can be adjusted to increase of decrease granularity
bins = int((df1_sorted['TimeDiff'].max() - df1_sorted['TimeDiff'].min()) / bin_size)
sns.histplot(df1_sorted['TimeDiff'], bins=bins)
plt.xlabel('Time Difference Between Transactions (Minutes)')
plt.ylabel('Average Frequency per Day')
plt.title('Average Daily Frequency of Time Differences Between Transactions')
plt.xlim(0, df1_sorted['TimeDiff'].quantile(0.95))  # Limit x-axis to exclude rare extreme values if necessary
plt.show()

In [ ]:
##PER MINUTE B: GAMMA##
# Ensure all time differences are positive (gamma distribution cannot have zero or negative values)
# Replace TimeDiff values of 0 or smaller with a small number (e.g., 0.001 seconds)
df1_sorted['TimeDiff'] = np.where(df1_sorted['TimeDiff'] <= 0, 0.001, df1_sorted['TimeDiff'])
#df1_sorted = df1_sorted[df1_sorted['TimeDiff'] > 0] #not good, since you will not take the arrivals on the same time with you

# Fit the empirical data to a gamma distribution, fixing the location at 0
params_gamma = stats.gamma.fit(df1_sorted['TimeDiff'], floc=0)

# Generate points on the x axis suitable for the range of your data
x_gamma = np.linspace(start=0, stop=df1_sorted['TimeDiff'].max(), num=10000)

# Plot the histogram of the empirical data
plt.figure(figsize=(15, 6))
sns.histplot(df1_sorted['TimeDiff'], bins=bins, kde=False, stat='density', label='Empirical Data')

# Plot the PDF of the fitted gamma distribution
pdf_gamma = stats.gamma.pdf(x_gamma, *params_gamma)
plt.plot(x_gamma, pdf_gamma, label='Fitted Gamma Distribution', color='orange')

# Finalize the plot
plt.xlabel('Time Difference Between Transactions (Minutes)')
plt.ylabel('Density')
plt.title('Average Daily Frequency of Time Differences Between Transactions with Fitted Gamma Distribution')
plt.xlim(0, df1_sorted['TimeDiff'].quantile(0.95))  # Limit x-axis to exclude rare extreme values if necessary
plt.legend()
plt.show()

In [ ]:
##NORMAL AND EXPONENTIAL DISTRIBUTION##
# Fit the empirical data to theoretical distributions
params_norm = stats.norm.fit(df1_sorted['TimeDiff'])
params_expon = stats.expon.fit(df1_sorted['TimeDiff'])

# Plot the histogram of the empirical data
plt.figure(figsize=(15, 6))
sns.histplot(df1_sorted['TimeDiff'], bins=bins, kde=False, stat='density', label='Empirical Data')

# Generate points on the x axis:
x = np.linspace(0, df1_sorted['TimeDiff'].quantile(0.95), 1000)

# Plot the PDF of the fitted normal distribution
pdf_norm = stats.norm.pdf(x, *params_norm)
plt.plot(x, pdf_norm, label='Fitted Normal Distribution', color='red')

# Plot the PDF of the fitted exponential distribution
pdf_expon = stats.expon.pdf(x, *params_expon)
plt.plot(x, pdf_expon, label='Fitted Exponential Distribution', color='green')

# Finalize the plot
plt.xlabel('Time Difference Between Transactions (Minutes)')
plt.ylabel('Density')
plt.title('Average Daily Frequency of Time Differences Between Transactions')
plt.xlim(0, df1_sorted['TimeDiff'].quantile(0.95))  # Limit x-axis to exclude rare extreme values if necessary
plt.legend()
plt.show()

In [ ]:
##NORMAL AND EXPONENTIAL DISTRIBUTION FIT RESULTS##

# Perform the Kolmogorov-Smirnov test for goodness of fit
ks_statistic_norm, p_value_norm = stats.kstest(time_diff_data, 'norm', args=params_norm)
ks_statistic_expon, p_value_expon = stats.kstest(time_diff_data, 'expon', args=params_expon)

# Print the test results
print(f'Normal Distribution KS Statistic: {ks_statistic_norm}, P-value: {p_value_norm}')
print(f'Exponential Distribution KS Statistic: {ks_statistic_expon}, P-value: {p_value_expon}')

In [ ]:
#Histogram arrival times TOTAL FREQUENCY AGGREGATION
# Plotting the histogram per hour
plt.figure(figsize=(10, 6))
plt.hist(df1['ArrivalHour'], bins=24, range=(0,24), edgecolor='black')
plt.title('Histogram of Arrival Times')
plt.xlabel('Hour of the Day')
plt.ylabel('Frequency')
plt.xticks(range(0, 25, 1))
plt.grid(axis='y', alpha=0.75)
plt.show()

In [ ]:
# Plotting the histogram per minute
plt.figure(figsize=(12, 6))
plt.hist(df1['ArrivalMinute'], bins=24*60, range=(0, 24*60), edgecolor='black')  # 24 hours * 60 minutes
plt.title('Histogram of Arrival Times Per Minute')
plt.xlabel('Minute of the Day')
plt.ylabel('Frequency')
plt.xticks(range(0, 24*60+1, 60), labels=[f'{h:02d}:00' for h in range(25)])  # Label every hour
plt.grid(axis='y', alpha=0.75)
plt.show()


In [ ]:
data = df1['ArrivalMinute']
print(data)

In [ ]:


def plot_distribution(df, title_suffix):
    arrival_counts = df['ArrivalHour'].value_counts().sort_index()
    departure_counts = df['DepartureHour'].value_counts().sort_index()
    
    colors = sns.color_palette("Greens", len(arrival_counts))
    sorted_colors_arrival = [colors[i] for i in arrival_counts.argsort()]
    sorted_colors_departure = [colors[i] for i in departure_counts.argsort()]
    
    width = 0.4  # Width of the bars
    hours = range(24)
    arrival_positions = [hour - width/2 for hour in hours]
    departure_positions = [hour + width/2 for hour in hours]
    
    plt.figure(figsize=(12, 5))
    
    plt.bar(arrival_positions, arrival_counts.reindex(hours, fill_value=0), color=sorted_colors_arrival, width=width, label='Arrival', align='center')
    plt.bar(departure_positions, departure_counts.reindex(hours, fill_value=0), color=sorted_colors_departure, width=width, label='Departure', align='center')
    
    plt.title(f'Average Distribution over 24 Hours ({title_suffix})')
    plt.xlabel('Hour of Day')
    plt.ylabel('Count')
    plt.xticks(hours)
    plt.legend()
    plt.tight_layout()



In [ ]:
# Plot distributions for weekdays and weekends
plot_distribution(df_weekdays, 'Weekdays')
plot_distribution(df_weekends, 'Weekends')


